<a href="https://colab.research.google.com/github/SiriN1234/Python_Learning_google_colab/blob/main/chapter_5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 : 혼자 공부하는 머신러닝+딥러닝

# 트리의 앙상블

- LightGBM 기억
  + GBM --> XGBoost --> LightGBM
  + 참고 1. 모델 개발 속도가 빨라졌나
  + 참고 2. 모델의 성능이 좋아졌나
- TabNet (2019)
  + 딥러닝 컨셉 이해 필요

## 랜덤 포레스트

- 결정 트리 나무를 500개 심기
- 최종적인 결정은 투표 방식
  + 나무-1 : 양성
  + 나무-2 : 음성
  + 나무-3 : 양성
  + ...
  + 나무-500 : 양성
  + 합산해서 결과 도출

# 데이터 불러오기, 데이터 셋 분리

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, 
                                                                      target, 
                                                                      test_size=0.2, 
                                                                      random_state=42)

- p. 267
  + cross_validate() 교차 검증 수행
  + RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 모든 CPU 코어를 사용하는 것이 좋음
  + return_train_score 매개변수를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환
  + 훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이함

In [3]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


- 과대적합
- 특성 중요도 출력

In [4]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 자체적으로 모델을 평가하는 점수를 얻을 수 있음
- 부트스트랩 샘플에 포함되지 않고 남는 샘플을 OOB 샘플이라 함
- OOB 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있음
- oob_score 매개변수를 True로 지정하면 사용 가능

In [5]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 그레이디언트 부스팅

- 이전 트리의 오차를 보완하는 방식으로 사용
- 깊이가 얕은 트리를 사용
- 학습을 매개변수로 속도를 조정
- 단점 : 속도가 느림

In [7]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [8]:
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


- 흐름
  + 0. 데이터 전처리 / 시각화
  + 1. 기본 모형으로 전체 흐름을 설계
  + 2. 여러 모형으로 비교 대조
  + 3. 교차 검증, 하피어파라미터 성능 비교
  + ...
  + 1등하는 그날까지

# 히스토그램 기반 그레이디언트 부스팅

- 정형 데이터를 다루는 머신러닝 알고리즘 중 가장 인기가 높은 알고리즘
- 입력 특성을 256개 구간으로 나눔
  + 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용
  + 입력에 누락된 특성이 있더라도 이를 따로 전처리할 필요가 없음


In [12]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score = True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


- 특성 중요도 확인

In [13]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats = 10, random_state = 42, n_jobs = -1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


- 테스트 세트의 특성 중요도 확인

In [14]:
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats = 10, random_state = 42, n_jobs = -1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


- 테스트 세트의 성능 확인

In [16]:
hgb.score(test_input, test_target)

0.8723076923076923

# XGBoost
- 그레디언트 부스팅 알고리즘을 구현한 라이브러리

In [20]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist', random_state = 42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score = True)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


# LightGBM

In [19]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score = True, n_jobs = -1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293


# 마무리

## 키워드로 끝내는 핵심 포인트

- **앙상블 학습**은 더 좋은 예측 결과를 만들기 위해 여러 개의 모델을 훈련하는 머신러닝 알고리즘을 말합니다.
- **랜덤 포레스트**는 대표적인 결정 트리 기반의 앙상블 학습 방법입니다. 부트스트랩 샘플을 사용하고 랜덤하게 일부 특성을 선택하여 트리를 만드는 것이 특징입니다.
- **엑스트라 트리**는 랜덤 포레스트와 비슷하게 결정 트리를 사용하여 앙상블 모델을 만들지만 부트스트랩 샘플을 사용하지 않습니다. 대신 랜덤하게 노드를 분할해 과대적합을 감소시킵니다.
- **그레이디언트 부스팅**은 랜덤 포레스트나 엑스트라 트리와 달리 결정 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법입니다. 이런 이유로 훈련 속도가 조금 느리지만 더 좋은 성능을 기대할 수 있습니다. 그레이디언트 부스팅의 속도를 개선한 것이 **히스토그램 기반 그레이디언트 부스팅**이며 안정적인 결과와 높은 성능으로 매우 인기가 높습니다

## 핵심 패키지와 함수

### scikit-learn

- **RandomForestClassifier**는 랜덤 포레스트 분류 클래스입니다.
  + n_estimators 매개변수는 앙상블을 구성할 트리의 개수를 지정합니다. 기본값은 100입니다.
  + criterion 매개변수는 불순도를 지정하며 기본값은 지니 불순도를 의미하는 'gini'이고 'entropy'를 선택하여 엔트로피 불순도를 사용할 수 있습니다.
  + max_depth는 트리가 성장할 최대 깊이를 지정합니다. 기본값은 None으로 지정하면 리프노드가 순수하거나 min_samples_split보다 샘플 개수가 적을 때까지 성장합니다.
  + min_samples_split은 노드를 나누기 위한 최소 샘플의 개수입니다. 기본값은 2입니다.
  + max_features 매개변수는 최적의 분할을 위해 탐색할 특성의 개수를 지정합니다. 기본값은 auto로 특성 개수의 제곱근입니다.
  + bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정합니다. 기본값은 True입니다.
  + oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정합니다. 기본값은 False입니다.
  + n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정합니다. 기본값은 1로 하나의 코어를 사용합니다. -1로 지정하면 시스템에 있는 모든 코어를 사용합니다.

- **ExtraTreesClassifier**는 엑스트라 트리 분류 클래스입니다.
  + n_estimators, criterion, max_depth, min_samples_split, max_features 매개변수는 랜덤 포레스트와 동일합니다.
  + bootstrap 매개변수는 부트스트랩 샘플을 사용할지 지정합니다. 기본값은 False입니다.
  + oob_score는 OOB 샘플을 사용하여 훈련한 모델을 평가할지 지정합니다. 기본값은 False입니다.
  + n_jobs 매개변수는 병렬 실행에 사용할 CPU 코어 수를 지정합니다. 기본값은 1로 하나의 코어를 사용합니다. -1로 지정하면 시스템에 있는 모든 코어를 사용합니다.

- **GradientBoostingClassifier**는 그레이디언트 부스팅 분류 클래스입니다.
  + loss 매개변수는 손실 함수를 지정합니다. 기본값은 로지스틱 손실 함수를 의미하는'deviance'입니다.
  + learning_rate 매개변수는 트리가 앙상블에 기여하는 정도를 조절합니다. 기본값은 0.1입니다.
  + n_estimators 매개변수는 부스팅 단계를 수행하는 트리의 개수입니다. 기본값은 100입니다.
  + subsample 매개변수는 사용할 훈련 세트의 샘플 비율을 지정합니다. 기본값은 1.0입니다.
  + max_depth 매개변수는 개별 회귀 트리의 최대 깊이입니다. 기본값은 3입니다.

- **HistGradientBoostingClassifier**는 히스토그램 기반 그레이디언트 부스팅 분류 클래스입니다.
  + learning_rate 매개변수는 학습률 또는 감쇠율이라고 합니다. 기본값은 0.1이며 1.0이면 감쇠가 전혀 없습니다.
  + max_iter는 부스팅 단계를 수행하는 트리의 개수입니다. 기본값은 100입니다.
  + max_bins는 입력 데이터를 나눌 구간의 개수입니다. 기본값은 255이며 이보다 크게 지정할 수 없습니다. 여기에 1개의 구간이 누락된 값을 위해 추가됩니다.